In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv("/Users/jitendra/Desktop/titanic/train.csv")
df_test = pd.read_csv("/Users/jitendra/Desktop/titanic/test.csv")

In [3]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
#'Survived' is the target variable
target = df_train['Survived']
features_train = df_train.drop(['Survived'],axis=1)

In [5]:
target

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [6]:
#identify categorical and continuous variables and separate them
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [9]:
categorical = df_train.columns[df_train.dtypes == 'object']
categorical = list(categorical)
categorical

['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

In [10]:
continuous = df_train.columns[df_train.dtypes != 'object']
continuous = list(continuous)

In [11]:
#we model only using the train and validation data
df_train[continuous].describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [12]:
df_train=df_train.drop(['PassengerId','Name','Ticket'],axis=1)
df_test=df_test.drop(['PassengerId','Name','Ticket'],axis=1)

In [13]:
#because Cabin has a lot of missing values
df_train=df_train.drop(['Cabin'],axis=1)
df_test=df_test.drop(['Cabin'],axis=1)

In [14]:
categorical = df_train.columns[df_train.dtypes == 'object']
categorical = list(categorical)
continuous = df_train.columns[df_train.dtypes != 'object']
continuous = [col for col in continuous if col != 'Survived']
continuous = list(continuous)

In [15]:
#impute categorical missing values with mode
#for both train and test datasets
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
X_train_categorical_imputed = imputer.fit_transform(df_train[categorical])
X_train_categorical_imputed
X_test_categorical_imputed = imputer.transform(df_test[categorical])
X_test_categorical_imputed

array([['male', 'Q'],
       ['female', 'S'],
       ['male', 'Q'],
       ['male', 'S'],
       ['female', 'S'],
       ['male', 'S'],
       ['female', 'Q'],
       ['male', 'S'],
       ['female', 'C'],
       ['male', 'S'],
       ['male', 'S'],
       ['male', 'S'],
       ['female', 'S'],
       ['male', 'S'],
       ['female', 'S'],
       ['female', 'C'],
       ['male', 'Q'],
       ['male', 'C'],
       ['female', 'S'],
       ['female', 'C'],
       ['male', 'C'],
       ['male', 'S'],
       ['female', 'S'],
       ['male', 'C'],
       ['female', 'C'],
       ['male', 'S'],
       ['female', 'C'],
       ['male', 'C'],
       ['male', 'S'],
       ['male', 'C'],
       ['male', 'S'],
       ['male', 'S'],
       ['female', 'S'],
       ['female', 'S'],
       ['male', 'C'],
       ['male', 'C'],
       ['female', 'S'],
       ['female', 'S'],
       ['male', 'S'],
       ['male', 'S'],
       ['male', 'C'],
       ['male', 'S'],
       ['male', 'S'],
       ['female', 'S']

In [16]:
continuous

['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [17]:
#impute continuous missing values with median
#for both train and test datasets
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
X_train_continuous_imputed = imputer.fit_transform(df_train[continuous])
X_train_continuous_imputed
X_test_continuous_imputed = imputer.transform(df_test[continuous])
X_test_continuous_imputed

array([[ 3.    , 34.5   ,  0.    ,  0.    ,  7.8292],
       [ 3.    , 47.    ,  1.    ,  0.    ,  7.    ],
       [ 2.    , 62.    ,  0.    ,  0.    ,  9.6875],
       ...,
       [ 3.    , 38.5   ,  0.    ,  0.    ,  7.25  ],
       [ 3.    , 28.    ,  0.    ,  0.    ,  8.05  ],
       [ 3.    , 28.    ,  1.    ,  1.    , 22.3583]])

In [18]:
X_train_features_imputed = pd.concat([pd.DataFrame(X_train_categorical_imputed),pd.DataFrame(X_train_continuous_imputed)],axis=1)
X_test_features_imputed = pd.concat([pd.DataFrame(X_test_categorical_imputed),pd.DataFrame(X_test_continuous_imputed)],axis=1)

In [19]:
X_train_features_imputed.columns = df_train[categorical+continuous].columns
X_test_features_imputed.columns = df_test[categorical+continuous].columns

In [20]:
X_train_features_imputed

,Sex,Embarked,Pclass,Age,SibSp,Parch,Fare
0,male,S,3.0,22.0,1.0,0.0,7.2500
1,female,C,1.0,38.0,1.0,0.0,71.2833
2,female,S,3.0,26.0,0.0,0.0,7.9250
3,female,S,1.0,35.0,1.0,0.0,53.1000
4,male,S,3.0,35.0,0.0,0.0,8.0500
...,...,...,...,...,...,...,...
886,male,S,2.0,27.0,0.0,0.0,13.0000
887,female,S,1.0,19.0,0.0,0.0,30.0000
888,female,S,3.0,28.0,1.0,2.0,23.4500
889,male,C,1.0,26.0,0.0,0.0,30.0000


In [21]:
from sklearn.preprocessing import OneHotEncoder


# Concatenate train and test data
combined_data = pd.concat([X_train_features_imputed[categorical], X_test_features_imputed[categorical]], axis=0)

categorical_columns = combined_data.select_dtypes(include=['object']).columns

# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# Fit and transform the encoder on the combined data
encoded_data = encoder.fit_transform(combined_data[categorical_columns])

# Convert the encoded data into a DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_columns))

# Split back into train and test data
train_data_encoded = encoded_df[:len(X_train_features_imputed[categorical])]
test_data_encoded = encoded_df[len(X_train_features_imputed[categorical]):]

/Users/jitendra/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [22]:
train_data_encoded

,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.0,1.0,0.0,0.0,1.0
1,1.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...
886,0.0,1.0,0.0,0.0,1.0
887,1.0,0.0,0.0,0.0,1.0
888,1.0,0.0,0.0,0.0,1.0
889,0.0,1.0,1.0,0.0,0.0


In [23]:
test_data_encoded

,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
891,0.0,1.0,0.0,1.0,0.0
892,1.0,0.0,0.0,0.0,1.0
893,0.0,1.0,0.0,1.0,0.0
894,0.0,1.0,0.0,0.0,1.0
895,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...
1304,0.0,1.0,0.0,0.0,1.0
1305,1.0,0.0,1.0,0.0,0.0
1306,0.0,1.0,0.0,0.0,1.0
1307,0.0,1.0,0.0,0.0,1.0


In [24]:
X_train_features_encoded = X_train_features_imputed.drop(categorical,axis=1)
X_train_features_encoded

,Pclass,Age,SibSp,Parch,Fare
0,3.0,22.0,1.0,0.0,7.2500
1,1.0,38.0,1.0,0.0,71.2833
2,3.0,26.0,0.0,0.0,7.9250
3,1.0,35.0,1.0,0.0,53.1000
4,3.0,35.0,0.0,0.0,8.0500
...,...,...,...,...,...
886,2.0,27.0,0.0,0.0,13.0000
887,1.0,19.0,0.0,0.0,30.0000
888,3.0,28.0,1.0,2.0,23.4500
889,1.0,26.0,0.0,0.0,30.0000


In [25]:
X_test_features_encoded = X_test_features_imputed.drop(categorical,axis=1)
X_test_features_encoded

,Pclass,Age,SibSp,Parch,Fare
0,3.0,34.5,0.0,0.0,7.8292
1,3.0,47.0,1.0,0.0,7.0000
2,2.0,62.0,0.0,0.0,9.6875
3,3.0,27.0,0.0,0.0,8.6625
4,3.0,22.0,1.0,1.0,12.2875
...,...,...,...,...,...
413,3.0,28.0,0.0,0.0,8.0500
414,1.0,39.0,0.0,0.0,108.9000
415,3.0,38.5,0.0,0.0,7.2500
416,3.0,28.0,0.0,0.0,8.0500


In [26]:
df_final_train = pd.concat([X_train_features_encoded,train_data_encoded],axis=1)
df_final_train

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3.0,22.0,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,1.0
1,1.0,38.0,1.0,0.0,71.2833,1.0,0.0,1.0,0.0,0.0
2,3.0,26.0,0.0,0.0,7.9250,1.0,0.0,0.0,0.0,1.0
3,1.0,35.0,1.0,0.0,53.1000,1.0,0.0,0.0,0.0,1.0
4,3.0,35.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
886,2.0,27.0,0.0,0.0,13.0000,0.0,1.0,0.0,0.0,1.0
887,1.0,19.0,0.0,0.0,30.0000,1.0,0.0,0.0,0.0,1.0
888,3.0,28.0,1.0,2.0,23.4500,1.0,0.0,0.0,0.0,1.0
889,1.0,26.0,0.0,0.0,30.0000,0.0,1.0,1.0,0.0,0.0


In [27]:
X_test_features_encoded

,Pclass,Age,SibSp,Parch,Fare
0,3.0,34.5,0.0,0.0,7.8292
1,3.0,47.0,1.0,0.0,7.0000
2,2.0,62.0,0.0,0.0,9.6875
3,3.0,27.0,0.0,0.0,8.6625
4,3.0,22.0,1.0,1.0,12.2875
...,...,...,...,...,...
413,3.0,28.0,0.0,0.0,8.0500
414,1.0,39.0,0.0,0.0,108.9000
415,3.0,38.5,0.0,0.0,7.2500
416,3.0,28.0,0.0,0.0,8.0500


In [28]:
test_data_encoded = test_data_encoded.reset_index(drop=True)
test_data_encoded

,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.0,1.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...
413,0.0,1.0,0.0,0.0,1.0
414,1.0,0.0,1.0,0.0,0.0
415,0.0,1.0,0.0,0.0,1.0
416,0.0,1.0,0.0,0.0,1.0


In [29]:
df_final_test = pd.concat([X_test_features_encoded,test_data_encoded],axis=1)
df_final_test

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3.0,34.5,0.0,0.0,7.8292,0.0,1.0,0.0,1.0,0.0
1,3.0,47.0,1.0,0.0,7.0000,1.0,0.0,0.0,0.0,1.0
2,2.0,62.0,0.0,0.0,9.6875,0.0,1.0,0.0,1.0,0.0
3,3.0,27.0,0.0,0.0,8.6625,0.0,1.0,0.0,0.0,1.0
4,3.0,22.0,1.0,1.0,12.2875,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
413,3.0,28.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,1.0
414,1.0,39.0,0.0,0.0,108.9000,1.0,0.0,1.0,0.0,0.0
415,3.0,38.5,0.0,0.0,7.2500,0.0,1.0,0.0,0.0,1.0
416,3.0,28.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,1.0


In [30]:
#Standardization for both train and test datasets
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
scaled_features_train = scaler.fit_transform(df_final_train[continuous])

scaled_features_df_train = pd.DataFrame(scaled_features_train, columns=df_final_train[continuous].columns)

scaled_features_test = scaler.transform(df_final_test[continuous])

scaled_features_df_test = pd.DataFrame(scaled_features_test, columns=df_final_test[continuous].columns)

In [31]:
df_final_train2 = df_final_train.drop(continuous,axis=1)
df_final_train2 = pd.concat([df_final_train2,scaled_features_df_train],axis=1)
df_final_test2 = df_final_test.drop(continuous,axis=1)
df_final_test2 = pd.concat([df_final_test2,scaled_features_df_test],axis=1)

In [32]:
df_final_train2

,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass,Age,SibSp,Parch,Fare
0,0.0,1.0,0.0,0.0,1.0,0.0,-0.461538,1.0,0.0,-0.312011
1,1.0,0.0,1.0,0.0,0.0,-2.0,0.769231,1.0,0.0,2.461242
2,1.0,0.0,0.0,0.0,1.0,0.0,-0.153846,0.0,0.0,-0.282777
3,1.0,0.0,0.0,0.0,1.0,-2.0,0.538462,1.0,0.0,1.673732
4,0.0,1.0,0.0,0.0,1.0,0.0,0.538462,0.0,0.0,-0.277363
...,...,...,...,...,...,...,...,...,...,...
886,0.0,1.0,0.0,0.0,1.0,-1.0,-0.076923,0.0,0.0,-0.062981
887,1.0,0.0,0.0,0.0,1.0,-2.0,-0.692308,0.0,0.0,0.673281
888,1.0,0.0,0.0,0.0,1.0,0.0,0.000000,1.0,2.0,0.389604
889,0.0,1.0,1.0,0.0,0.0,-2.0,-0.153846,0.0,0.0,0.673281


In [33]:
from sklearn.svm import SVC

In [38]:
svc = SVC() 

In [39]:
#Does Cross Validation and hyperparameter search in one go

from sklearn.model_selection import GridSearchCV
param_grid_svc = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}
# Use grid search
grid = GridSearchCV(svc, param_grid_svc, cv=5)
grid.fit(df_final_train2, target)
grid.best_params_

{'C': 1, 'kernel': 'rbf'}

In [48]:
grid.best_score_

0.8193019898311468

In [41]:
svc = SVC(C=1,kernel='rbf')

In [46]:
svc.fit(df_final_train2, target)

SVC(C=1)

In [47]:
y_pred = svc.predict(df_final_test2)

y_pred

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,